In [ ]:
import numpy as np
import pandas as  pd
import matplotlib.pyplot as plt 
from sklearn import *
import math
from matplotlib.pyplot import figure
import matplotlib.pyplot as plt
import scipy
from sklearn import linear_model as lm  # Used for solving linear regression problems
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm
import cmath
import math
import datetime as dt
import matplotlib.dates as mdates
from sklearn.neural_network import MLPRegressor


In [ ]:
def get_hourly_purchases(df_agg_hours,timestamp_agg_hours, ItemID):
    
    """
    Generates a dataframe with the chosen ItemID and its purchases every hour 
    Arguments:
        :df_agg_hours: df with the aggregated purchases by timestamp (day+hour) and ItemID
        :ItemID: Size of the plot, tuple
        :timestamp_agg_hours: Support dictionary that has all the hours during the month to be filled
        Returns:
        Hourly Purchases of ItemID == ID in a DataFrame.
    """
    # add 0 sales hours for product
    support = dict(zip(timestamp_agg_hours,[0]*len(timestamp_agg_hours)))
    filler = df_agg_hours[df_agg_hours['ItemID']==ItemID]
    
    
    for i in range(len(filler)):
        support[filler.iloc[i,0]] = filler.iloc[i,-1]


    df = pd.DataFrame(support.items(), columns=['timestamp', 'quantity']).set_index('timestamp')

    return df

In [ ]:
def get_daily_purchases(df_agg_days, timestamp_agg_days, ItemID):
    
    """
    Generates a dataframe with the chosen ItemID and its purchases every hour 
    Arguments:
        :df_agg_days: df with the aggregated purchases by day and ItemID
        :ItemID: Size of the plot, tuple
        Returns:
        Daily Purchases of ItemID == ID in a DataFrame.
    """
    
    # add 0 sales days for product

    support = dict(zip(timestamp_agg_days,[0]*len(timestamp_agg_days)))
    filler = df_agg_days[df_agg_days['ItemID']==ItemID]
    
    for i in range(len(filler)):
        support[filler.iloc[i,0]] = filler.iloc[i,-1]

    df = pd.DataFrame(support.items(), columns=['DayDate', 'quantity']).set_index('DayDate')
    
    return df

In [ ]:
def plot_trend_item(name_product, series, daily, size=(20, 8), interval = 10):
    """
    Plots the trend of an item groupped in the span of days of hours.
    Arguments:
        :name_product: NameID to be shown in title
        :series: DF with 2 columns: one has to be date times (%Y-%m-%d or %Y-%m-%d %H) and the other the 
            actual frequency counting of that product
        :size: Size of the plot, tuple
        :daily: boolean, True if the aggregation is daily based, False if hourly
        Returns:
        Plot of this Series
    """
    # Define plot space
    
    # define if hourly
    time_span = "Daily" if daily else "Hourly"

    plt.figure(figsize=size)    
    plt.title(time_span + " purchase trend of item = " + name_product, size=20)    
    plt.plot(series, label='sales', color="blue")   
    
    # Tick Freq
    plt.gca().xaxis.set_major_locator(mdates.DayLocator(interval=interval))


    plt.xlabel("Time (" + time_span + ")", size=20)
    plt.ylabel("Units Sales", size=20)

    
    plt.xticks(rotation = 90) # Rotates X-Axis Ticks by 90-degrees

    plt.grid(True)




In [ ]:
def plot_predictions_1step(y_pred, y_true, name_item, train_size, daily=1, size=(25,8), interval = 2):
     
    """
    Plots predictions of the forecasting model and the background truth of the corresponding timestamps
    Arguments:
        :y_pred: Series of the predictions and Time Stamps as index
        :y_true: Series of the True and Time Stamps as index. dimensions must much y_pred
        :name_item: String, name of the product for the title 
        :train_size: To plot vertical line in the burn-in period. Actually pass train-p because in the predictions 
        :there is a burn-in period used to learn the model params
        :daily: boolean, True if the aggregation is daily based, False if hourly (just title purposes)
        :size: Size of the plot, tuple
        Returns:
        Plot of this Series
    """
        
    # define if hourly
    time_span = "Daily" if daily else "Hourly"
    
    plt.title("1-step-ahead prediction for product = " + name_item, size=20)    
    plt.plot(y_pred, label='Sales Predicted', color="blue")   
    plt.plot(y_true, label='True Sales', color="brown")   
    
    # Tick Freq
    plt.gca().xaxis.set_major_locator(mdates.DayLocator(interval=interval))
    plt.xticks(rotation = 90) # Rotates X-Axis Ticks by 90-degrees

    plt.xlabel("Time (" + time_span + ")", size=20)
    plt.ylabel("Units Sales", size=20)

    # Burn-in vert line
    plt.axvline(x = train_size, color = 'red', linestyle= '--', label = 'burn-in period')

    
    plt.grid(True)

    
    plt.legend(fontsize='xx-large')
    plt.show()

In [ ]:
def predict_ar_1step(theta, y_target):
    """Predicts the value y_t for t = p+1, ..., n, for an AR(p) model, based on the data in y_target using
    one-step-ahead prediction.

    :param theta: array (p,), AR coefficients, theta=(a1,a2,...,ap).
    :param y_target: array (n,), the data points used to compute the predictions.
    :return y_pred: array (n-p,), the one-step predictions (\hat y_{p+1}, ...., \hat y_n) 
    """

    n = len(y_target)
    p = theta.shape[1]
    
    # Number of steps in prediction
    m = n-p
    y_pred = np.zeros(m+1)
    
    # Go predict
    for i in range(m):
        y_pred[i] =  (np.flip(y_target[i:i+p]) * theta).sum()
        
    return y_pred

In [ ]:
def fit_ar(y, p):
    """Fits an AR(p) model. The loss function is the sum of squared errors from t=p+1 to t=n.

    :param y: array (n,), training data points
    :param p: int, AR model order
    :return theta: array (p,), learnt AR coefficients
    """

    # Number of training data points
    n = y.shape[0]
    
    # Construct the regression matrix
    Phi = np.zeros((n-p, p))
    for j in range(p):
        Phi[:,j] = y[(p-(j+1)): (n-(j+1)), 0] 
    
    # Drop the first p values from the target vector y
    yy = y[p:]  # yy = (y_{t+p+1}, ..., y_n)

    # Here we use fit_intercept=False since we do not want to include an intercept term in the AR model
    regr = lm.LinearRegression(fit_intercept=False)
    regr.fit(Phi,yy)    

    return regr.coef_

In [ ]:

def fit_nar(y, p, nhl):
    """Fits an AR(p) model. The loss function is the sum of squared errors from t=p+1 to t=n.

    :param y: array (n,), training data points
    :param p: int, AR model order
    :return theta: array (p,), learnt AR coefficients
    """

    # Number of training data points
    n = y.shape[0]
    
    # Construct the regression matrix
    Phi = np.zeros((n-p, p))# <COMPLETE THIS LINE>
    for j in range(p):
        Phi[:,j] = y[(p-(j+1)): (n-(j+1)), 0] # <COMPLETE THIS LINE>
    
    # Drop the first p values from the target vector y
    yy = y[p:].ravel()  # yy = (y_{t+p+1}, ..., y_n)

    # Here we use fit_intercept=False since we do not want to include an intercept term in the AR model
    regr =  MLPRegressor(hidden_layer_sizes = nhl, max_iter = 1000).fit(Phi, yy)    

    return regr

In [ ]:
def plot_ts(ts, plot_ma=True, plot_intervals=True, window=30,
            figsize=(15,5), interval=5):
    '''
    Plot ts with rolling mean and 95% confidence interval with rolling std.
    :parameter    
      :param ts: pandas Series    
      :param window: num - for rolling stats
      :param plot_ma: bool - whether plot moving average
      :param plot_intervals: bool - whether plot upper and lower bounds
      :interval: frequency of X axis Ticks
    '''
    # Mean and STD Rolling
    rolling_mean = ts.rolling(window=window).mean()    
    rolling_std = ts.rolling(window=window).std()
    
    # Plot Space
    plt.figure(figsize=figsize)    
    plt.title(ts.name)    
    plt.plot(ts[window:], label='Actual values', color="black")   
    plt.gca().xaxis.set_major_locator(mdates.DayLocator(interval=interval))

    # Paint the series
    if plot_ma:        
        plt.plot(rolling_mean, 'g', label='MA'+str(window),
               color="red")    
        
    # Plot the bounds 
    if plot_intervals:
        lower_bound = rolling_mean - (1.96 * rolling_std)
        upper_bound = rolling_mean + (1.96 * rolling_std)
    
    # filler
    
    plt.fill_between(x=ts.index, y1=lower_bound, y2=upper_bound,
                    color='lightskyblue', alpha=0.4)
    
    plt.xticks(rotation = 90) # Rotates X-Axis Ticks by 90-degrees

    plt.legend(loc='best')
    plt.grid(True)
    plt.show()

In [ ]:

def find_outliers(ts, name_item=None, perc=0.01, figsize=(15,5), interval=3, plot=0):
    
    '''
    Find outliers using sklearn unsupervised support vetcor machine.
        :parameter
        :param ts: pandas Series
        :param perc: float - percentage of outliers to look for
        :return
            dtf with raw ts, outlier 1/0 (yes/no), numeric index
    '''
    # SVM Params #
    #gamma# inverse of the radius of influence of samples selected by model as support vectors
    #C# The larger the smaller the Margin accepted. The lower the larger the margin 
    
    
    ## fit svm Radial Basis Function
    scaler = preprocessing.StandardScaler()
    ts_scaled = scaler.fit_transform(ts.values.reshape(-1,1))
    model = svm.OneClassSVM(nu=perc, kernel="rbf", gamma=0.01)
    model.fit(ts_scaled)
    
    ## dtf output
    dtf_outliers = ts.to_frame(name="ts")
    dtf_outliers["index"] = range(len(ts))
    dtf_outliers["outlier"] = model.predict(ts_scaled)
    dtf_outliers["outlier"] = dtf_outliers["outlier"].apply(lambda
                                              x: 1 if x==-1 else 0)
    if plot:
        ## plot
        fig, ax = plt.subplots(figsize=figsize)
        ax.set(title="Outliers detection: found "
               +str(sum(dtf_outliers["outlier"]==1))) + ' of product = ' + name_item
        ax.plot(dtf_outliers["ts"],
                color="black")
        ax.scatter(x=dtf_outliers[dtf_outliers["outlier"]==1]["index"],
                   y=dtf_outliers[dtf_outliers["outlier"]==1]['ts'],
                   color='red')
        plt.gca().xaxis.set_major_locator(mdates.DayLocator(interval=interval))
        plt.xticks(rotation = 90) # Rotates X-Axis Ticks by 90-degrees

        ax.grid(True)
        plt.show()
    
    return dtf_outliers

In [ ]:
def find_outliers_plot(dtf_outliers, name_item= None, figsize=(15,5), interval=3, filter_threshold=None):
        '''
        Plots the DF object returned from the find_outliers function
            :parameter
            :param dtf_outliers: pandas Dataframe. Must have columns "outlier" (bool) and 'ts' (float), index must be 
                timestamp 
            :param perc: float - percentage of outliers to look for
            :name_item: String for title purpose
            :interval: frequency of X axis Ticks

            :return
                dtf with raw ts, outlier 1/0 (yes/no), numeric index
        '''
        # Copy the df not to modify inplace
        dtf_outliers = dtf_outliers.copy()
        fig, ax = plt.subplots(figsize=figsize)
        
        # Paint the signal
        ax.plot(dtf_outliers["ts"],
                color="black")
        
        # Filter anomalies to get the viral anomalies only
        if filter_threshold is not None:
            thresh = dtf_outliers.ts.mean() + dtf_outliers.ts.std() * filter_threshold
            
            idx = dtf_outliers[dtf_outliers['ts'] < thresh ].index
            dtf_outliers.loc[idx, 'outlier'] = 0

        # Scatter points
        ax.scatter(x=dtf_outliers[dtf_outliers["outlier"]==1]["index"],
                   y=dtf_outliers[dtf_outliers["outlier"]==1]['ts'],
                   color='red')
        
        # Title + Ticks
        ax.set(title="Outliers detection: found " +str(sum(dtf_outliers["outlier"]==1))+ ' of product = ' + name_item) 

        plt.gca().xaxis.set_major_locator(mdates.DayLocator(interval=interval))
        plt.xticks(rotation = 90) # Rotates X-Axis Ticks by 90-degrees

        ax.grid(True)
        
        plt.show()

In [ ]:
## Creating user-defined function for arranging the results obtained from model into readable format

def inspect(results):
    '''
    Rearranges the result object from the rules function in apriori algorithm (apyori)
        :parameter
        :results: rules object out of  apriori() function (apyori package)  

        :return
            zipped object ready for the pd.Dataframe transformation to be visualised
    '''
    lhs         = [tuple(result[2][0][0])[0] for result in results]
    rhs         = [tuple(result[2][0][1])[0] for result in results]
    supports    = [result[1] for result in results]
    confidences = [result[2][0][2] for result in results]
    lifts       = [result[2][0][3] for result in results]
    
    return list(zip(lhs, rhs, supports, confidences, lifts))


In [3]:
from statsmodels.tsa.stattools import adfuller

def check_stationarity(series):
    # Copied from https://machinelearningmastery.com/time-series-data-stationary-python/

    result = adfuller(series.values)

    print('ADF Statistic: %f' % result[0])
    print('p-value: %f' % result[1])
    print('Critical Values:')
    for key, value in result[4].items():
        print('\t%s: %.3f' % (key, value))

    if (result[1] <= 0.05) & (result[4]['5%'] > result[0]):
        print("\u001b[32mStationary\u001b[0m")
    else:
        print("\x1b[31mNon-stationary\x1b[0m")